In [135]:
import pandas as pd
import os

dir = r'data/energia/ceny/'

files = os.listdir(dir)


In [136]:
files = sorted(files)

In [197]:
from datetime import datetime, timedelta

def format(raw_data):
    df = raw_data.copy()
    a_mask = df['Godzina'].apply(lambda x: 'A' in str(x))
    df.drop(index=df[a_mask].index, inplace=True)
    df['Godzina'] = df['Godzina'].astype(str)
    mask = (df['Godzina'] == '24')
    df['Godzina'] = df['Godzina'].replace({'24': '0'}).apply(lambda x: x if len(x) > 1 else f'0{x}')
    tt = pd.to_datetime(df[['Data', 'Godzina']].astype(str).agg(' '.join, axis=1), format="%Y-%m-%d %H")
    tt.loc[mask] = tt.loc[mask].apply(lambda x: x + timedelta(days=1))
    df['Time'] = tt
    df.drop(columns=['Data', 'Godzina'], inplace=True)
    df.rename(columns={'RCE': 'Value'}, inplace=True)
    df['Value'] = df['Value'].replace({' ': '', '\xa0': '', ',': '.'}, regex=True)
    df['Value'] = pd.to_numeric(df['Value'])
    df.set_index('Time', inplace=True)
    return df


df = pd.concat([pd.read_csv(dir + file, sep=';', parse_dates=['Data'], encoding='windows-1252') for file in files], ignore_index=True)
df

,Data,Godzina,RCE
0,2018-01-01,1,"117,02"
1,2018-01-01,2,"114,00"
2,2018-01-01,3,"99,02"
3,2018-01-01,4,"92,07"
4,2018-01-01,5,"82,44"
...,...,...,...
43075,2022-12-31,20,"337,30"
43076,2022-12-31,21,"289,23"
43077,2022-12-31,22,"182,31"
43078,2022-12-31,23,"159,65"


In [199]:
df = format(df)

In [206]:
df.to_csv('data/energy_prices.csv')

In [170]:
format(df)

ValueError: Unable to parse string "1 021.57" at position 32270